### NAO correlations with energy variables ###

Exploring how well the NAO correlates with energy variables on seasonal to decadal timescales during the winter (ONDJFM, DJFM, or DJF). Using the following datasets:

* CLEARHEADS - ERA5-derived energy time series, includes offshore wind in EEZs and Heating Degree Days.
* ERA5 - reanalysis product for deriving the NAO indices at different timescales.
* ENTSO-E - shorter observed time series of capacity factors and other energy variables. For ground truthing the CLEARHEADS data above.

In [1]:
# -*- coding: utf-8 -*-
%matplotlib inline
%load_ext autoreload
%autoreload 2

# Import local modules
import sys
import os
import glob

# Import third-party modules
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import xarray as xr
import cartopy.crs as ccrs
import cartopy.feature as cfeature
import cdsapi
import xesmf as xe

### Downloading ERA5 data ###

For calculating the NAO index, we want to query the CDS API for ERA5 data:

* From 1950-2023
* For ONDJFM
* Monthly-means

*Note - this data should be regridded before comparison with the CLEARHEADS/ENTSO-E data*

In [2]:
# Set up a new client
c = cdsapi.Client()

In [3]:
# Set up the dictionary for the ERA5 request
era5_request_dict = {
    'variable': 'mean_sea_level_pressure',
    'product_type': 'monthly_averaged_reanalysis',
    'year': [x for x in map(str, range(1950, 2023))],
    'month': [1, 2, 3, 10, 11, 12],
    'format': 'netcdf',
    'time': '00:00'
}

# Print the request dictionary
print(era5_request_dict)

{'variable': 'mean_sea_level_pressure', 'product_type': 'monthly_averaged_reanalysis', 'year': ['1950', '1951', '1952', '1953', '1954', '1955', '1956', '1957', '1958', '1959', '1960', '1961', '1962', '1963', '1964', '1965', '1966', '1967', '1968', '1969', '1970', '1971', '1972', '1973', '1974', '1975', '1976', '1977', '1978', '1979', '1980', '1981', '1982', '1983', '1984', '1985', '1986', '1987', '1988', '1989', '1990', '1991', '1992', '1993', '1994', '1995', '1996', '1997', '1998', '1999', '2000', '2001', '2002', '2003', '2004', '2005', '2006', '2007', '2008', '2009', '2010', '2011', '2012', '2013', '2014', '2015', '2016', '2017', '2018', '2019', '2020', '2021', '2022'], 'month': [1, 2, 3, 10, 11, 12], 'format': 'netcdf', 'time': '00:00'}


In [4]:
# Set up the target directory
target_dir = '/gws/nopw/j04/canari/users/benhutch/ERA5'

# Assert that the target directory exists
assert os.path.exists(target_dir)

# Assert that the target directory is not empty
assert len(os.listdir(target_dir)) > 0

# Set up the target file
target_file = os.path.join(target_dir, 'era5_mslp_monthly_1950_2022_ONDJFM.nc')

# Print the target file
print(target_file)

# If the target file does not exist, download the data
if not os.path.exists(target_file):
    c.retrieve(
        'reanalysis-era5-single-levels',
        era5_request_dict,
        target_file)
else:
    print('The target file already exists: {}'.format(target_file))

/gws/nopw/j04/canari/users/benhutch/ERA5/era5_mslp_monthly_1950_2022_ONDJFM.nc


2024-02-19 13:58:55,035 INFO Welcome to the CDS
2024-02-19 13:58:55,036 INFO Sending request to https://cds.climate.copernicus.eu/api/v2/resources/reanalysis-era5-single-levels
2024-02-19 13:58:55,207 INFO Request is queued
